In [1]:
from keras.models import Sequential # using sequential model
from keras.layers import Conv2D # for the convolutional layers and 2D photographs
from keras.layers import MaxPooling2D
from keras.layers import Flatten # flatten image
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator # process image data

C:\Users\tajff\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
def make_model(parameters):
    check_boxes=parameters["check_box"]
    neuron=parameters["neurons"]

    layers=[]
    for check_box, neurons in zip(check_boxes, neuron):
        if check_box:
            layers.append(neurons)
    classifier = Sequential() # tworzy obiekt o informacjach sieci neuronowej
    classifier.add(Conv2D(32,(3,3), input_shape = (64, 64, 3), activation = 'relu')) # create the input layer
    
    classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2,2))) # mapping data through layers and now we can reduce the data to two sets.
    classifier.add(Flatten()) # flatten to single dimensions

    for units in layers:
        classifier.add(Dense(units = 128, activation = 'relu')) # take the values that were flattened for this layer
        classifier.add(Dense(units = 1, activation = 'sigmoid')) # filter the data to a single output hence the unit is 1 (so its 0-1)

    return classifier



In [3]:
parameters={"check_box":[True, False, True],"neurons":[5, 10, 15],
            "training_path":'C:/Users/tajff/Documents/Korepetycjepyt/Cat-Dog-Predictor-with-NeuralNetwork-master/trainData/',
            "test_path":'C:/Users/tajff/Documents/Korepetycjepyt/Cat-Dog-Predictor-with-NeuralNetwork-master/testData/', 
            "optimizer":'adam', "loss" : 'binary_crossentropy'}

In [4]:
def train_model(parameters):
    model=make_model(parameters)
    optimizer=parameters["optimizer"]
    loss=parameters["loss"]
    model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])
# compile the neural network, the optimizer is the reverse propagation, the loss and metrics compute the error
    
    training_path=parameters["training_path"] 
    training_data_generator = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,
                                                 zoom_range = 0.2, horizontal_flip = True)
                                                # rescale, so the colurs are between 0 and 1
    
    
    training_set = training_data_generator.flow_from_directory(training_path, 
                                                               target_size = (64,64), batch_size = 32, class_mode = 'binary')
    
    test_path=parameters["test_path"] 
    test_data_generator = ImageDataGenerator(rescale=1./255)
    test_set = training_data_generator.flow_from_directory(test_path, 
                                                           target_size = (64,64), batch_size = 32, class_mode = 'binary')
    model.fit_generator(training_set, steps_per_epoch = 4000, epochs = 15, 
                             validation_data = test_set, validation_steps = 100 )
# go through each picture 4000 times, adjust values to change steps per epoch, epochs and the validation steps for training the neural network
    return model

In [5]:
classifier=train_model(parameters)

Found 25000 images belonging to 1 classes.
Found 12500 images belonging to 1 classes.
Epoch 1/15
  31/4000 [..............................] - ETA: 22:01 - loss: 0.6547 - acc: 0.9677

KeyboardInterrupt: 

In [11]:
import numpy as np
from keras.preprocessing import image
# make a prediction from a test image
test_image = image.load_img('C:/Users/tajff/Documents/Korepetycjepyt/Cat-Dog-Predictor-with-NeuralNetwork-master/testData/test1/661.jpg',
                            target_size = (64,64))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)

NameError: name 'classifier' is not defined